In [1]:
import os
import sys
import requests
from PIL import Image
from io import BytesIO
import csv
from tqdm import tqdm

In [2]:
# Ruta al archivo CSV
csv_file_path = 'dataset_images_diffusion.csv'

# Ruta de la carpeta donde se guardarán las imágenes procesadas
output_folder_path = 'dataset/fake'

# Crear la carpeta si no existe
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [3]:
# Función para procesar cada imagen
def process_image(image_url, output_folder):
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()  # Lanza una excepción si la respuesta es un error

        # Recorta y guarda la imagen
        with Image.open(BytesIO(response.content)) as image:
            cropped_image = image.crop((0, 0, image.width, image.height - 50))
            file_name = os.path.basename(image_url)
            save_path = os.path.join(output_folder, file_name)
            cropped_image.save(save_path)

    except requests.exceptions.HTTPError as e:
        print(f"Error HTTP al descargar {image_url}: {e}", file=sys.stderr)

In [4]:
with open(csv_file_path, newline='') as f:
    # Contar el total de líneas para la barra de progreso
    total_lines = sum(1 for line in f)
    f.seek(0)  # Regresar al inicio del archivo
    
    image_urls = csv.reader(f)
    for image_url in tqdm(image_urls, total=total_lines, desc='Descargando imágenes', unit='imagen', file=sys.stdout):
        if image_url:  # Si la fila no está vacía
            process_image(image_url[0], output_folder_path)

Descargando imágenes:   1%|          | 6/1000 [00:07<21:39,  1.31s/imagen]


KeyboardInterrupt: 